In [49]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm.auto import tqdm, trange 

In [50]:
data = pd.read_csv(r"C:\Users\d5767\VS code projects\business_letter_clusterer\letters.csv", sep=';')

In [51]:
data.head(3)

,from,to,salutation,subject_line,part_1,part_2,part_3,complimentary_close
0,NaN,NaN,Dear Charles Milton,NaN,I would like to invite you to a seminar that I...,The 3D Technologies Seminar held at the Moscow...,I am enclosing 3 tickets for you. I hope that ...,Best regards
1,NaN,NaN,Dear Mr. Smith:,NaN,Thank you for your conscientious service. All ...,Ted McCracken and Bob Smiley have delivered th...,Doing business with your organization is a ple...,NaN
2,NaN,"Mr R. Cliff,\nHomemakers Ltd., 54-59 Riverside...","Dear Mr Cliff,",NaN,Thank you for your last delivery. You will be ...,A number of my customers have been asking abou...,"I have enclosed an order, No. B1463, in antici...","Yours sincerely,"


In [52]:
# 3 раздела по 4 новостей в каждом
topics = ["part_1", "part_2", "part_3"]
news_count =  3 #3000

In [53]:
df_res = pd.DataFrame()




for topic in tqdm(topics):
    df_topic = data[topic] # [:news_count]
    for df_data in df_topic:
        df_res = df_res.append([[df_data, topics.index(topic)]], ignore_index=True)
    

df_res

100%|██████████| 3/3 [00:00<00:00, 49.18it/s]


,0,1
0,I would like to invite you to a seminar that I...,0
1,Thank you for your conscientious service. All ...,0
2,Thank you for your last delivery. You will be ...,0
3,We were impressed by the selection of sweaters...,0
4,We have learnt from the Russian Trade Delegati...,0
5,"With reference to my letter of 10 August, I en...",0
6,I saw your advertisement for a Business Journa...,0
7,I'm writing to you in regard of your enquiry. ...,0
8,The 3D Technologies Seminar held at the Moscow...,1
9,Ted McCracken and Bob Smiley have delivered th...,1


In [54]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in
             nltk.word_tokenize(sent)] #разбиение на токены
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token): #удаление всего, кроме букв
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens] #стемминг
    return stems   

In [55]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\d5767\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
stopwords = nltk.corpus.stopwords.words("english") 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_featur = 200_000

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10_000,
                                  min_df=0.01, stop_words=stopwords,
                                  use_idf=True, tokenizer=token_and_stem)

In [57]:
texts = df_res[0]
type(texts)

pandas.core.series.Series

In [58]:
#%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(df_res[0])
tfidf_matrix

C:\Users\d5767\AppData\Roaming\Python\Python38\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'s", 'abov', 'ani', 'becaus', 'befor', 'could', 'doe', 'dure', 'might', 'must', "n't", 'need', 'onc', 'onli', 'ourselv', 'sha', 'themselv', 'veri', 'whi', 'wo', 'would', 'yourselv'] not in stop_words.
  warnings.warn(


<24x330 sparse matrix of type '<class 'numpy.float64'>'
	with 449 stored elements in Compressed Sparse Row format>

In [59]:
#подготовка данных для обучения и тестов
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf_matrix, df_res[1], test_size=0.2, random_state=0)


In [60]:
x_train = x_train.toarray()
x_test = x_test.toarray()



In [73]:
#обучение классификатора
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

clf_list = [LinearDiscriminantAnalysis(), KNeighborsClassifier(), GaussianNB(), DecisionTreeClassifier()]

for clf in clf_list:
    print(type(clf))
    clf.fit(x_train, y_train)



<class 'sklearn.discriminant_analysis.LinearDiscriminantAnalysis'>
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
<class 'sklearn.naive_bayes.GaussianNB'>
<class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [115]:
for clf in clf_list:
    print(clf.score(tfidf_matrix.toarray(), df_res[1]))


0.5833333333333334
0.5416666666666666
0.9166666666666666
0.875


In [79]:
#from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
model = DecisionTreeClassifier()
c = cross_val_score(model, tfidf_matrix, df_res[1] , cv=5)
print(c)
print(c.mean())

[0.4 0.4 0.4 1.  0.5]
0.54


In [121]:
from sklearn.metrics import accuracy_score
for d in range(1, 20):
    model = DecisionTreeClassifier(max_depth=d)
    model.fit(x_train, y_train)
    p = model.predict(x_test)
    print(f'max_depth = {d} \t accuracy_score = {accuracy_score(y_test, p)}')
    # print(f'f1_score: {(y_test, p)}')
    # print()

    # Получение оценки hoLd-out
#tree_pred = tree.predict(X_holdout)
#accur = accuracy_score(y_holdout, tree_pred) print(accur)

max_depth = 1 	 accuracy_score = 0.0
max_depth = 2 	 accuracy_score = 0.4
max_depth = 3 	 accuracy_score = 0.0
max_depth = 4 	 accuracy_score = 0.6
max_depth = 5 	 accuracy_score = 0.6
max_depth = 6 	 accuracy_score = 0.6
max_depth = 7 	 accuracy_score = 0.6
max_depth = 8 	 accuracy_score = 0.2
max_depth = 9 	 accuracy_score = 0.4
max_depth = 10 	 accuracy_score = 0.4
max_depth = 11 	 accuracy_score = 0.6
max_depth = 12 	 accuracy_score = 0.8
max_depth = 13 	 accuracy_score = 0.6
max_depth = 14 	 accuracy_score = 0.8
max_depth = 15 	 accuracy_score = 0.6
max_depth = 16 	 accuracy_score = 0.2
max_depth = 17 	 accuracy_score = 0.2
max_depth = 18 	 accuracy_score = 0.6
max_depth = 19 	 accuracy_score = 0.2


In [113]:
from sklearn import tree
model = DecisionTreeClassifier(max_depth=4)
model.fit(x_train, y_train)
tree.export_graphviz(model, class_names=['1','2','3'], out_file='iris_tree.dot', filled=True, rounded=True)

# Генерируея графическое представление дерева tree. export_graphviz(tree_grid. best_estimator_.,
 


In [ ]:
#сохранение модели 
import pickle

with open("modelGaussianNB.pkl", mode="wb") as f:
    pickle.dump(clf_list[2],f)


In [ ]:
#сохранение модели 
import pickle

with open("tfidf_vectorizer.pkl", mode="wb") as f:
    pickle.dump(tfidf_vectorizer,f)

In [ ]:
#загрузка модели
import pickle

with open("modelGaussianNB.pkl", mode="rb") as f:
    load_clf  = pickle.load(f)
    
load_clf

GaussianNB()

In [ ]:
#загрузка модели
import pickle

with open("tfidf_vectorizer.pkl", mode="rb") as f:
    load_tfidf_vectorizer  = pickle.load(f)
    
load_tfidf_vectorizer

TfidfVectorizer(max_df=0.8, max_features=10000, min_df=0.01,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...],
                tokenizer=<function token_and_stem at 0x05100FA0>)

In [65]:

out_df = pd.DataFrame(df_res[1])
for clf in clf_list:
    
    out_df = pd.concat([out_df,pd.DataFrame(clf.predict(tfidf_matrix.toarray()),  columns = ['out'])], axis=1)
out_df


,1,out,out,out,out
0,0,1,0,0,0
1,0,0,0,0,0
2,0,2,2,0,0
3,0,0,0,0,0
4,0,0,0,0,0
5,0,0,1,0,0
6,0,0,2,0,0
7,0,0,0,0,0
8,1,1,0,1,1
9,1,1,0,1,1


test_blok = """I have enclosed an order, No. B1463, in anticipation of you agreeing, and as there /sno particular hurry for the units, you could send them along with your next delivery."""

tfidf_matrix = load_tfidf_vectorizer.transform([test_blok])

load_clf.predict(tfidf_matrix.toarray())[0]



#расстояние между векторами(cos) <- для графиков взаимного расположения заголовков
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix) # вычитание из 1 чтобы не было отриц значений
dist.shape